---
title: IDs from STEREO
format:
  html:
    code-fold: true
output-file: stereo.html
---

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create stearo
```

To get STEREO-A state data, run `kedro run --to-outputs=sta.primary_state_rtn_1h`

To get candidates data, run `kedro run --from-inputs=sta.feature_1s --to-outputs=candidates.sta_1s`

In [ ]:
#| default_exp pipelines/stearo/pipeline

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| code-summary: import all the packages needed for the project
#| export
#| output: hide
import os

from fastcore.utils import *
from fastcore.test import *
from ids_finder.utils.basic import *
from ids_finder.core import *

import polars as pl
import pandas
import numpy as np

from datetime import timedelta
from loguru import logger

from typing import Callable, Dict

os.environ['SPEDAS_DATA_DIR'] = f"{os.environ['HOME']}/data"

In [ ]:
#| export
from ids_finder.core import extract_features

#### `Kerdo`

In [ ]:
#| export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

In [ ]:
#| eval: false
from ids_finder.utils.basic import load_catalog

catalog = load_catalog()
catalog.list()

[10/16/23 17:08:34] WARNING  KedroDeprecationWarning: 'ParquetDataSet' has been renamed to          ]8;id=392624;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=471812;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             'ParquetDataset', and the alias will be removed in Kedro-Datasets                     
                             2.0.0                                                                                 
                                                                                                                   

                    WARNING  FutureWarning: Defining the 'layer' attribute at the top level is      ]8;id=22531;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=834025;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             deprecated and will be removed in Kedro 0.19.0. Please move 'layer'                   
                             inside the 'metadata' -> 'kedro-viz' attributes. See                                  
                             https://docs.kedro.org/en/latest/visualisation/kedro-viz_visualisation                
                             .html#visualise-layers for more information.                                          
                                                                                                                   

                    WARNING  KedroDeprecationWarning: 'CSVDataSet' has been renamed to              ]8;id=788510;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=955333;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             'CSVDataset', and the alias will be removed in Kedro-Datasets 2.0.0                   
                                                                                                                   

[10/16/23 17:08:35] WARNING  KedroDeprecationWarning: 'GenericDataSet' has been renamed to          ]8;id=298216;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=711730;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             'GenericDataset', and the alias will be removed in Kedro-Datasets                     
                             2.0.0                                                                                 
                                                                                                                   


[
    'JNO_index',
    'jno.raw_jno_ss_se_1sec',
    'jno.inter_jno_ss_se_1s',
    'jno.raw_state',
    'model.raw_jno_ss_se_1min',
    'model.preprocessed_jno_ss_se_1min',
    'candidates_all_1s',
    'sta.raw_sta_state',
    'parameters',
    'params:tau',
    'params:jno_start_date',
    'params:jno_end_date',
    'params:jno_1s_params',
    'params:jno_1s_params.bcols',
    'params:jno_1s_params.data_resolution',
    'params:jno.extract_params',
    'params:jno.extract_params.bcols',
    'params:jno.extract_params.data_resolution',
    'params:sta.extract_params',
    'params:sta.extract_params.bcols',
    'params:sta.extract_params.data_resolution'
]

## Dataset Overview

In [ ]:
jno_start_date = catalog.load('params:jno_start_date')
jno_end_date = catalog.load('params:jno_end_date')
print(jno_start_date, jno_end_date)
trange = [jno_start_date, jno_end_date]
test_trange = ["2011-08-25", "2011-09-01"]

                    INFO     Loading data from 'params:jno_start_date' (MemoryDataset)...       ]8;id=770440;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=228825;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py#492\492]8;;\

                    INFO     Loading data from 'params:jno_end_date' (MemoryDataset)...         ]8;id=897474;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=969811;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py#492\492]8;;\

2011-08-25 2016-06-30


### Magnetic field data

STEREO magnetic field is already in RTN coordinates, so no need to transform it.

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

#### Downloading data

In [ ]:
#| export
import pyspedas

In [ ]:
# | export
def download_mag_data(
    start: str = None,
    end: str = None,
    probe: str = "a",
) -> Iterable[str]:
    trange = [start, end]
    files = pyspedas.stereo.mag(trange, downloadonly=True, probe=probe)
    return files

#### Preprocessing data

In [ ]:
from ids_finder.utils.basic import cdf2pl
from pipe import select
from typing import Iterable

In [ ]:
# | export
def preprocess_mag_data(
    raw_data: Iterable[str] = None,
    ts: str = "1s",  # time resolution
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Downsample the data to a given time resolution
    - Applying naming conventions for columns
    """
    every = pandas.Timedelta(ts)
    period = 2 * every

    df: pl.LazyFrame = pl.concat(raw_data | pmap(cdf2pl, var_name="BFIELD"))

    return (
        df.pipe(resample, every=every, period=period)
        .rename(
            {
                "BFIELD_0": "b_r",
                "BFIELD_1": "b_t",
                "BFIELD_2": "b_n",
                "BFIELD_3": "b_mag",
            }
        )
        .collect()
    )

#### Processing data

In [ ]:
#| export
def process_mag_data(
    raw_data: pl.DataFrame,
    ts: str = None,  # time resolution
    coord: str = None,
) -> Dict[str, pl.DataFrame]:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Partitioning data, for the sake of memory
    """
    return partition_data_by_year(raw_data)

#### Pipeline

In [ ]:
#| export
def create_mag_data_pipeline(
    sat_id,
    ts: str = "1s",  # time resolution,
    tau: str = "60s",  # time window
    **kwargs,
) -> Pipeline:
    node_download_mag_data = node(
        download_mag_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
        ),
        outputs=f"raw_mag",
        name=f"download_{sat_id.upper()}_magnetic_field_data",
    )

    node_preprocess_mag_data = node(
        preprocess_mag_data,
        inputs=dict(
            raw_data=f"raw_mag",
        ),
        outputs=f"inter_mag_{ts}",
        name=f"preprocess_{sat_id.upper()}_magnetic_field_data",
    )

    node_process_mag_data = node(
        process_mag_data,
        inputs=f"inter_mag_{ts}",
        outputs=f"primary_mag_rtn_{ts}",
        name=f"process_{sat_id.upper()}_magnetic_field_data",
    )

    node_extract_features = node(
        extract_features,
        inputs=[f"primary_mag_rtn_{ts}", "params:tau", "params:extract_params"],
        outputs=f"feature_tau_{tau}",
        name=f"extract_{sat_id}_features",
    )

    nodes = [
        node_download_mag_data,
        node_preprocess_mag_data,
        node_process_mag_data,
        node_extract_features,
    ]

    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
            "params:tau": "params:tau",
        },
    )

    return pipelines

### State data
We use [COHOWeb](https://omniweb.gsfc.nasa.gov/coho/) for state data.

See [STEREO ASCII merged data](https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/aareadme_sta) and one sample file [here](https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/stereoa2011.asc

Plasma in RTN (Radial-Tangential-Normal) coordinate system
- Proton Flow Speed, km/sec
- Proton Flow Elevation Angle/Latitude, deg.
- Proton Flow Azimuth Angle/Longitude, deg.
- Proton Density, n/cc
- Proton Temperature, K)

Notes
- Note1: There is a big gap 2014/12/16 - 2015/07/20 in plasma data
- Note2: There is a big gap 2015/03/21 - 2015/07/09 and 2015/10/27 - 2015/11/15 in mag data
- Note that for missing data, fill values consisting of a blank followed by 9's which together constitute the format are used

#### Get state data

In [ ]:
#| export
import pooch

In [ ]:
# | export
def download_stereo_state_data(tstart, tend):
    download = partial(pooch.retrieve, known_hash=None)

    start_time = pandas.Timestamp(tstart)
    end_time = pandas.Timestamp(tend)

    url = "https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/merged/stereoa{year}.asc"

    files = list(
        range(start_time.year, end_time.year + 1)
        | select(lambda x: url.format(year=x))
        | select(download)
    )
    return files


headers = """Year
DOY
Hour
Radial Distance, AU
HGI Lat. of the S/C
HGI Long. of the S/C
IMF BR, nT (RTN)
IMF BT, nT (RTN)
IMF BN, nT (RTN)
IMF B Scalar, nT
SW Plasma Speed, km/s
SW Lat. Angle RTN, deg.
SW Long. Angle RTN, deg.
SW Plasma Density, N/cm^3
SW Plasma Temperature, K
1.8-3.6 MeV H flux,LET
4.0-6.0 MeV H flux,LET
6.0-10.0 MeV H flux, LET
10.0-12.0 MeV H flux,LET
13.6-15.1 MeV H flux, HET
14.9-17.1 MeV H flux, HET
17.0-19.3 MeV H flux, HET
20.8-23.8 MeV H flux, HET
23.8-26.4 MeV H flux, HET
26.3-29.7 MeV H flux, HET
29.5-33.4 MeV H flux, HET
33.4-35.8 MeV H flux, HET
35.5-40.5 MeV H flux, HET
40.0-60.0 MeV H flux, HET
60.0-100.0 MeV H flux, HET
0.320-0.452 MeV H flux, SIT
0.452-0.64 MeV H flux, SIT
0.640-0.905 MeV H flux, SIT
0.905-1.28 MeV H flux, SIT
1.280-1.81 MeV H flux, SIT
1.810-2.56 MeV H flux, SIT
2.560-3.62 MeV H flux, SIT"""

columns_position = [
    "Radial Distance, AU",
    "HGI Lat. of the S/C",
    "HGI Long. of the S/C",
]

columns_plasma = [
    "SW Plasma Speed, km/s",
    "SW Lat. Angle RTN, deg.",
    "SW Long. Angle RTN, deg.",
    "SW Plasma Density, N/cm^3",
    "SW Plasma Temperature, K",
]
columns_with_interested_features = columns_position + columns_plasma

columns_name_mapping = {
    "SW Plasma Speed, km/s": "sw_speed",
    "SW Lat. Angle RTN, deg.": "sw_elevation",
    "SW Long. Angle RTN, deg.": "sw_azimuth",
    "SW Plasma Density, N/cm^3": "sw_density",
    "SW Plasma Temperature, K": "sw_temperature",
    "Radial Distance, AU": "radial_distance",
}


def get_state_data(
    tstart, tend, columns=columns_with_interested_features
) -> pl.DataFrame:
    files = download_stereo_state_data(tstart, tend)

    labels = headers.split("\n")
    missing_values = ["999.99", "9999.9", "9999999."]

    df = pl.concat(
        files
        | pmap(
            pandas.read_csv,
            delim_whitespace=True,
            names=labels,
            na_values=missing_values,
        )
        | select(pl.from_pandas)
    )

    return df.select(
        columns,
        time=(
            pl.datetime(pl.col("Year"), month=1, day=1)
            + pl.duration(days=pl.col("DOY") - 1, hours=pl.col("Hour"))
        ).dt.cast_time_unit("ns"),
    ).rename(columns_name_mapping)

#### Processs state data

In [ ]:
#| export
def convert_state_to_rtn(df: pl.DataFrame) -> pl.DataFrame:
    """Convert state data to RTN coordinates
    """
    return df.with_columns(
        sw_vel_r=pl.col('sw_speed') * pl.col('sw_elevation').radians().cos() * pl.col('sw_azimuth').radians().cos(),
        sw_vel_t=pl.col('sw_speed') * pl.col('sw_elevation').radians().cos() * pl.col('sw_azimuth').radians().sin(),
        sw_vel_n=pl.col('sw_speed') * pl.col('sw_elevation').radians().sin(),
    ).drop(["sw_elevation", "sw_azimuth"])
    
def processs_state_data(df: pl.DataFrame) -> pl.DataFrame:
    return convert_state_to_rtn(df)

#### Pipelines

In [ ]:
# | export
def create_state_data_pipeline(**kwargs) -> Pipeline:
    sat_id = "sta"
    node_get_state_data = node(
        get_state_data,
        inputs=["params:start_date", "params:end_date"],
        outputs="inter_state_rtn_1h",
        name=f"get_{sat_id.upper()}_state_data",
    )

    node_processs_state_data = node(
        processs_state_data,
        inputs="inter_state_rtn_1h",
        outputs="primary_state_rtn_1h",
        name=f"process_{sat_id.upper()}_state_data",
    )

    nodes = [node_get_state_data, node_processs_state_data]
    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
        },
    )

    return pipelines

## Processing the whole data

### Pipelines

## Pipelines

In [ ]:
#| export
from ids_finder.candidates import create_candidate_pipeline

In [ ]:
# | export
def create_pipeline(**kwargs) -> Pipeline:
    sat_id = "sta"
    return (
        create_mag_data_pipeline(sat_id)
        + create_state_data_pipeline()
        + create_candidate_pipeline(sat_id)
    )

## Obsolete codes

NOTE: one can also use `speasy` to download data, however this is slower for `STEREO` data.

In [ ]:
%%markdown
sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)

Download data in a background thread

In [ ]:
%%markdown

@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, disable_proxy=True)
    logger.info("Data downloaded")
    
download_data(products, trange)

In [ ]:
import speasy as spz

In [ ]:
cda_tree: spz.SpeasyIndex = spz.inventories.tree.cda
product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN

logger.info(product.description)
logger.info(product.ID)
logger.info(product.BFIELD.CATDESC)
logger.info(product.BFIELD.spz_uid())

# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.
# spz.inventories.data_tree.cda.STEREO.STEREOA.IMPACT_MAG.STA_LB_MAG_RTN.description
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.MAGFLAGUC.CATDESC
spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.CATDESC
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.